This notebook extracts number of insurance issuers per county per year from the health insurance marketplace service area PUFs.

Here's the high level strategy:
- concatenate all service area data into one dataframe
- create a new dataframe of all the service areas that cover an entire state in a given year
- create a new dataframe where each of those service areas is given a row for each county in the state it covers
- gather all the service areas that only cover a specific county
- concatenate the dataframe of service areas that cover an entire state with the dataframe of those that cover only a specific county
- use this dataframe to count the number of distinct issuers per county per year

Read in the data. If we don't read in the County column as a string, pandas wil read it in as an int, messing up the values.

In [171]:
import pandas as pd

svArea2014 = pd.read_csv('Service_Area_PUF_2014.csv',  dtype={'County': str})
svArea2015 = pd.read_csv('Service_Area_PUF_2015.csv',  dtype={'County': str})
svArea2016 = pd.read_csv('Service_Area_PUF_2016.csv',  dtype={'County': str})
svArea2017 = pd.read_csv('service-area-2017.csv',  dtype={'County': str})

We only keep certain columns from the datasets and only want to investigate plans for individuals (vs. for small businesses) that aren't dental only plans. Unfortunately, the MarketCoverage and DentalOnly columns are missing many values, so we'll filter by what we know the values *shouldn't* be, but I'm sure some plans we don't want will sneak in.

Additionally, for some reason the 2014-2016 data the County column has a decimal and a 0 on the end of each value, so we remove that.

In [186]:
svArea2014.columns

Index(['BusinessYear', 'StateCode', 'IssuerId', 'SourceName', 'VersionNum',
       'ImportDate', 'IssuerId2', 'StateCode2', 'ServiceAreaId',
       'ServiceAreaName', 'CoverEntireState', 'County', 'PartialCounty',
       'ZipCodes', 'PartialCountyJustification', 'RowNumber', 'MarketCoverage',
       'DentalOnly'],
      dtype='object')

In [244]:
colsToKeep = [
    'BusinessYear', 
    'StateCode', 
    'ServiceAreaId',
    'ServiceAreaName', 
    'CoverEntireState', 
    'County', 
    'IssuerId']
def filterSvArea(df, DentalOnlyCol='DentalOnlyPlan'):
    df['County'] = df.County.str.replace('\.0', '')
    
    return df[(df.MarketCoverage != 'SHOP (Small Group)') 
              & (df[DentalOnlyCol] != 'Yes')
              & (~df.ServiceAreaName.str.contains('Dental', case=False)) 
              & (~df.ServiceAreaName.str.contains('Small', case=False))
              & (~df.ServiceAreaName.str.contains('Group', case=False))
              & (~df.ServiceAreaName.str.contains('Shop', case=False))
              & (~df.ServiceAreaName.str.contains('DHMO', case=False))
              & (~df.ServiceAreaName.str.contains('DPPO', case=False))
              & (~df.ServiceAreaName.str.contains('DDS', case=False))
              & (~df.ServiceAreaName.str.contains('DIC', case=False))
             ][colsToKeep]
   

svAreas = pd.concat([
            filterSvArea(svArea2014, DentalOnlyCol='DentalOnly'),
            filterSvArea(svArea2015, DentalOnlyCol='DentalOnly'), 
            filterSvArea(svArea2016),
            filterSvArea(svArea2017)
            ])

We presume that if a service area covers the entire state, its issuer is present in every county in that state. So that we can account for that later, we create a dataframe where each row corresponds to an issuer who had a service area that covered an entire state for the given business year.

In [245]:
svAreas[(svAreas.StateCode == "AZ") & (svAreas.BusinessYear == 2015) & (svAreas.CoverEntireState == "Yes")]

,BusinessYear,StateCode,ServiceAreaId,ServiceAreaName,CoverEntireState,County,IssuerId
199,2015,AZ,AZS001,Statewide,Yes,NaN,12303
200,2015,AZ,AZS001,AD AZ,Yes,NaN,13576
209,2015,AZ,AZS001,Arizona Service Area,Yes,NaN,18156
210,2015,AZ,AZS001,Service Area 1,Yes,NaN,20128
211,2015,AZ,AZS001,Arizona,Yes,NaN,21463
214,2015,AZ,AZS001,Arizona,Yes,NaN,24106
215,2015,AZ,AZS001,Arizona,Yes,NaN,30045
216,2015,AZ,AZS001,Service Area 1,Yes,NaN,30884
217,2015,AZ,AZS001,Arizona,Yes,NaN,31957
218,2015,AZ,AZS001,Arizona,Yes,NaN,33851


In [246]:
colsToKeep = ['BusinessYear', 'StateCode', 'IssuerId']
entireState = svAreas[svAreas.CoverEntireState == "Yes"]
entireState = entireState.groupby(['BusinessYear','StateCode', 'IssuerId']).count().reset_index()[colsToKeep]
entireState[(entireState.StateCode == "AZ") & (entireState.BusinessYear == 2015)].shape

(26, 3)

Read in the fips codes that are used to identify each county in the US. I downloaded them [here](https://www.census.gov/geo/reference/codes/cou.html). 

We concatenate the State and County component of the Fips code so we'll be able to join the dataframe with our service area data later.

For the join, we also need to duplicate the counties four times so that there is one row for each county for each business year.

In [226]:
dtypes = {1: str, 2: str}
colNames = {0: 'StateCode', 1:'StateFP', 2:'CountyFP', 3:'CountyName', 4:'ClassFP'}
fipsCodes = pd.read_csv('fips-codes.csv', header=None, dtype=dtypes).rename(columns=colNames)
fipsCodes['County'] = fipsCodes.StateFP.str.cat(fipsCodes.CountyFP)
numCounties = fipsCodes.shape[0]
fipsCodes = pd.concat([fipsCodes, fipsCodes, fipsCodes, fipsCodes])
fipsCodes['BusinessYear'] = 0
fipsCodes = fipsCodes.reset_index()

for i, year in enumerate([2014,2015,2016,2017]):
    begIndex = i * numCounties
    endIndex = begIndex + numCounties
    fipsCodes.loc[begIndex:endIndex, 'BusinessYear'] = year
fipsCodes.head()

,index,StateCode,StateFP,CountyFP,CountyName,ClassFP,County,BusinessYear
0,0,AL,01,001,Autauga County,H1,01001,2014
1,1,AL,01,003,Baldwin County,H1,01003,2014
2,2,AL,01,005,Barbour County,H1,01005,2014
3,3,AL,01,007,Bibb County,H1,01007,2014
4,4,AL,01,009,Blount County,H1,01009,2014


Join the fipsCode dataframe with the dataframe of issuers who cover an entire state. This way, we can associate an issuer who covers an entire state with each of the counties in that state.  

In [234]:
entireStateByCounties = entireState.merge(fipsCodes, on=['BusinessYear', 'StateCode'])
entireStateByCounties[entireStateByCounties.StateCode == "AZ" &].groupby('County').count().head()

,BusinessYear,StateCode,IssuerId,index,StateFP,CountyFP,CountyName,ClassFP
County,,,,,,,,
04001,73,73,73,73,73,73,73,73
04003,73,73,73,73,73,73,73,73
04005,73,73,73,73,73,73,73,73
04007,73,73,73,73,73,73,73,73
04009,73,73,73,73,73,73,73,73


Get servicesAreas/issuers who only cover specific counties. Note we're not distinguishing as to whether it covers and entire county or only part of it.

In [238]:
colsToKeep = ['BusinessYear', 'StateCode', 'IssuerId', 'County']
countyIssuer = (svAreas[(svAreas.County.notnull())]
               .groupby(['BusinessYear','StateCode', 'County', 'IssuerId'])
               .count()
               .reset_index()[colsToKeep])
countyIssuer[(countyIssuer.StateCode == "AZ") & (countyIssuer.BusinessYear == 2015)].shape

(45, 4)

In [251]:
countyOnly = countyIssuer.groupby(['BusinessYear', 'StateCode', 'County']).agg({'IssuerId': lambda x: x.nunique()}).reset_index()
countyOnly = countyOnly.rename(columns={'IssuerId': 'NumIssuers'})
gr = countyOnly.groupby(['BusinessYear', 'StateCode']).NumIssuers.mean()
gr.loc[2015]

StateCode
AK     1.000000
AL     1.000000
AR     2.000000
AZ     3.000000
FL     5.164179
GA     5.050314
IA     1.093750
IL     3.450980
IN     4.510870
KS     3.180952
LA     1.937500
ME     1.000000
MI     5.240964
MO     3.939130
MS     2.121951
MT     1.000000
NC     3.120000
ND     1.285714
NE     2.698925
NH     1.700000
NJ     3.238095
NM     1.151515
NV     1.882353
OH     3.602273
OK     4.727273
OR     4.027778
PA    13.701493
SC     2.152174
SD     1.375000
TN     3.263158
TX     3.401575
UT     2.068966
VA     2.828358
WI     3.888889
WY     1.000000
Name: NumIssuers, dtype: float64

In [95]:
for year in [svArea2014, svArea2015, svArea2016, svArea2017]:
    print(year[year.MarketCoverage.isnull()].StateCode.unique())

['PA' 'AK' 'AL' 'AZ' 'FL' 'GA' 'IL' 'IN' 'LA' 'MI' 'MO' 'MS' 'NC' 'ND' 'NJ'
 'OH' 'OK' 'SC' 'TN' 'TX' 'WI' 'WY' 'VA']
['PA' 'AK' 'AL' 'AZ' 'FL' 'GA' 'IN' 'LA' 'MO' 'MS' 'NC' 'ND' 'NJ' 'OK' 'SC'
 'TN' 'TX' 'WI' 'WY']
['NJ' 'AK' 'AL' 'AZ' 'FL' 'GA' 'IN' 'LA' 'MO' 'MS' 'NC' 'ND' 'OK' 'PA' 'TN'
 'SC' 'TX' 'WI' 'WY']
['AK' 'AL' 'AZ' 'FL' 'GA' 'IN' 'LA' 'MO' 'MS' 'NC' 'ND' 'NJ' 'TN' 'OK' 'SC'
 'TX' 'WI' 'WY']


Some of the counties in the service area data don't have the full five digit Fips code, so we need to fix that.

In [212]:
def fipsToFive(fips):
    fipsLen = len(fips)
    if fipsLen < 5:
        zerosNeeded = '0' * (5 - fipsLen)
        fips = zerosNeeded + fips
    return fips
countyIssuer['County'] = countyIssuer.County.apply(fipsToFive)

Concatenate the dataframe of issuers who cover an entire state with the dataframe of those who cover specific only counties.

In [213]:
entireAndCounties = pd.concat([countyIssuer, entireStateByCounties[colsToKeep]])
entireAndCounties.head()

,BusinessYear,StateCode,IssuerId,County
0,2014,AL,44580,01073
1,2014,AL,44580,01089
2,2014,AL,44580,01117
3,2014,AR,62141,05005
4,2014,AR,70525,05005


Now that we have one dataframe containing issuers who cover an entire state along with those who cover only specific counties, we can count the distinct number of issuers per county per year.

In [214]:
issuerCounts = entireAndCounties.groupby(['BusinessYear', 'StateCode', 'County']).agg({'IssuerId': lambda x: x.nunique()}).reset_index()
issuerCounts = issuerCounts.rename(columns={'IssuerId': 'NumIssuers'})

In [215]:
issuerCounts.to_csv('issuerCounts.csv', index=False)

In [216]:
issuerCounts[(issuerCounts.StateCode == "AL") & (issuerCounts.BusinessYear == 2015)].NumIssuers.mean()

13.044776119402986

In [222]:
issuerCounts[issuerCounts.StateCode == "AZ"]

,BusinessYear,StateCode,County,NumIssuers
171,2014,AZ,04001,16
172,2014,AZ,04003,15
173,2014,AZ,04005,16
174,2014,AZ,04007,16
175,2014,AZ,04009,15
176,2014,AZ,04011,15
177,2014,AZ,04012,15
178,2014,AZ,04013,18
179,2014,AZ,04015,16
180,2014,AZ,04017,16


In [247]:
meansPerYear = issuerCounts.groupby(['BusinessYear', 'StateCode']).NumIssuers.mean()
meansPerYear.loc[2015]

StateCode
AK    12.000000
AL    13.044776
AR     4.000000
AZ    28.666667
DE     3.000000
FL    19.686567
GA    23.050314
IA     1.141414
IL     4.676471
IN    21.510870
KS     3.180952
LA    16.937500
ME     3.000000
MI     6.168675
MO    16.939130
MS     7.121951
MT     4.000000
NC    17.120000
ND    11.094340
NE     2.698925
NH     4.900000
NJ    14.333333
NM     5.000000
NV     2.882353
OH     6.602273
OK    16.818182
OR     7.444444
PA    22.701493
SC    17.152174
SD     3.000000
TN    16.263158
TX    23.401575
UT     4.206897
VA     2.828358
WI    19.888889
WV     1.000000
WY    12.043478
Name: NumIssuers, dtype: float64

In [218]:
svArea2015[svArea2015.StateCode == "AZ"].ServiceAreaName.unique()

array(['Statewide', 'AD AZ', 'MetLife Preferred Dental Program',
       'DentalGuard Preferred PPO- AZ', 'Arizona Service Area',
       'Service Area 1', 'Arizona', 'HHP HMO.1', 'HHP HMO.3',
       'Statewide PPO', 'PPO', 'HMO - Maricopa', 'HMO - Pima',
       'Service Area One', 'Meritus Complete HMO',
       'Meritus HMO Maricopa MIHS', 'Meritus HMO Phoenix',
       'Meritus HMO Pima, Santa Cruz', 'Meritus HMO Mohave', 'PHP HMO.1',
       'Dental', 'Health Choice HMO.1', 'Health Choice HMO.2',
       'Health Choice HMO.3', 'Health Choice HMO.4', 'Service Area 2',
       'Service Area 3', 'OAMC_AZ_Banner Health',
       'Cigna Health and Life Insurance Company Statewide PPO',
       'Cigna Health and Life Insurance Company Statewide PPO - Dental',
       'UHM Arizona Service Area', 'CommunityCare', 'Statewide HMO',
       'ExcelCare', 'CommunityCare IFP', 'Arizona Statewide Service Area',
       'AZ FFS Exchange'], dtype=object)